In [2]:
import os
import torch
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from tqdm.auto import tqdm
import huggingface_hub as hf
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from typing import List, Dict, Union, Tuple
from transformers import AutoTokenizer, AutoModel

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 256)

plt.style.use('seaborn-v0_8')
load_dotenv()
hf.login(os.environ["HF_TOKEN"])
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
print("CUDA_VISIBLE_DEVICES:", os.environ["CUDA_VISIBLE_DEVICES"], "HF_HOME:", os.environ["HF_HOME"])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/mohsenfayyaz/.cache/huggingface/token
Login successful
CUDA_VISIBLE_DEVICES: 7 HF_HOME: /local1/mohsenfayyaz/.hfcache/


In [3]:
# DATASET = "re-docred_facebook--contriever-msmarco_7170.pkl"
DATASET = "re-docred_facebook--dragon-plus-query-encoder_7170.pkl"
# DATASET = "re-docred_OpenMatch--cocodr-base-msmarco_7170.pkl.gz"

hf.hf_hub_download(repo_id="Retriever-Contextualization/datasets", filename=f"results/{DATASET}", repo_type="dataset", local_dir="hf/")
df_raw = pd.read_pickle(f"./hf/results/{DATASET}")
print(df_raw.attrs)
df_raw.head(1)

{'model': 'facebook/dragon-plus-query-encoder', 'query_model': 'facebook/dragon-plus-query-encoder', 'context_model': 'facebook/dragon-plus-context-encoder', 'pooling': 'cls', 'dataset': 're-docred', 'corpus_size': 105925, 'eval': {'ndcg': {'NDCG@1': 0.47685, 'NDCG@3': 0.52523, 'NDCG@5': 0.53646, 'NDCG@10': 0.54955, 'NDCG@100': 0.58002, 'NDCG@1000': 0.59556}, 'map': {'MAP@1': 0.47685, 'MAP@3': 0.51341, 'MAP@5': 0.51959, 'MAP@10': 0.52496, 'MAP@100': 0.53058, 'MAP@1000': 0.53109}, 'recall': {'Recall@1': 0.47685, 'Recall@3': 0.55941, 'Recall@5': 0.58689, 'Recall@10': 0.62748, 'Recall@100': 0.77741, 'Recall@1000': 0.90349}, 'precision': {'P@1': 0.47685, 'P@3': 0.18647, 'P@5': 0.11738, 'P@10': 0.06275, 'P@100': 0.00777, 'P@1000': 0.0009}}}


,query_id,query,gold_docs,gold_docs_text,scores_stats,scores_gold,scores_1000,predicted_docs_text_10,id,title,vertexSet,labels,sents,split,label,label_idx,head_entity,tail_entity,head_entity_names,tail_entity_names,head_entity_longest_name,tail_entity_longest_name,head_entity_types,tail_entity_types,evidence_sent_ids,evidence_sents,head_entity_in_evidence,tail_entity_in_evidence,relation,relation_name,query_question,duplicate_titles_len,duplicate_titles,hit_rank,gold_doc,gold_doc_title,gold_doc_text,gold_doc_score,pred_doc,pred_doc_title,pred_doc_text,pred_doc_score,gold_doc_len,pred_doc_len,query_decompx_tokens,query_decompx_tokenizer_word_ids,query_decompx_cls_or_mean_pooled,query_decompx_tokens_dot_scores,query_decompx_decompx_last_layer_pooled,gold_doc_decompx_tokens,gold_doc_decompx_tokenizer_word_ids,gold_doc_decompx_cls_or_mean_pooled,gold_doc_decompx_tokens_dot_scores,gold_doc_decompx_decompx_last_layer_pooled,pred_doc_decompx_tokens,pred_doc_decompx_tokenizer_word_ids,pred_doc_decompx_cls_or_mean_pooled,pred_doc_decompx_tokens_dot_scores,pred_doc_decompx_decompx_last_layer_pooled
0,test0,When was Loud Tour published?,[Loud Tour],"{'Loud Tour': {'text': 'The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth stud...","{'len': 1000, 'max': 390.3378601074219, 'min': 377.525390625, 'std': 1.243663421340353, 'mean': 378.77503692626954, 'median': 378.4281463623047}",{'Loud Tour': 390.3378601074219},"{'Loud Tour': 390.3378601074219, 'Loud'n'proud': 385.71905517578125, 'Poetry Bus Tour': 385.4292907714844, 'Live &amp; Loud': 384.18218994140625, 'The Loudest Engine': 384.0265808105469, 'Young Wild Things Tour': 383.8572998046875, 'Guitar Rock Tour': ...","{'Loud Tour': {'text': 'The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the Americas and Europe , the tour was launched in support of Rihanna 's fifth stud...",test0,Loud Tour,"[[{'name': 'Loud', 'pos': [23, 24], 'sent_id': 1, 'type': 'MISC', 'global_pos': [41, 41], 'index': '0_0'}, {'name': 'Loud Tour', 'pos': [1, 3], 'sent_id': 6, 'type': 'MISC', 'global_pos': [128, 128], 'index': '0_1'}, {'name': 'Loud Tour', 'pos': [1, 3]...","[{'r': 'P577', 'h': 0, 't': 6, 'evidence': [1]}, {'r': 'P175', 'h': 0, 't': 2, 'evidence': [0, 1]}, {'r': 'P131', 'h': 10, 't': 8, 'evidence': [4]}, {'r': 'P17', 'h': 8, 't': 7, 'evidence': [3, 4]}, {'r': 'P17', 'h': 10, 't': 7, 'evidence': [3, 4]}, {'...","[[The, Loud, Tour, was, the, fourth, overall, and, third, world, concert, tour, by, Barbadian, recording, artist, Rihanna, .], [Performing, in, over, twenty, countries, in, the, Americas, and, Europe, ,, the, tour, was, launched, in, support, of, Rihan...",test,"{'r': 'P577', 'h': 0, 't': 6, 'evidence': [1]}",0,"[{'name': 'Loud', 'pos': [23, 24], 'sent_id': 1, 'type': 'MISC', 'global_pos': [41, 41], 'index': '0_0'}, {'name': 'Loud Tour', 'pos': [1, 3], 'sent_id': 6, 'type': 'MISC', 'global_pos': [128, 128], 'index': '0_1'}, {'name': 'Loud Tour', 'pos': [1, 3],...","[{'pos': [25, 26], 'type': 'TIME', 'sent_id': 1, 'name': '2010', 'global_pos': [43, 43], 'index': '6_0'}]","{Loud Tour, Loud}",{2010},Loud Tour,2010,{MISC},{TIME},[1],"[[Performing, in, over, twenty, countries, in, the, Americas, and, Europe, ,, the, tour, was, launched, in, support, of, Rihanna, 's, fifth, studio, album, Loud, (, 2010, ), .]]","[{'name': 'Loud', 'pos': [23, 24], 'sent_id': 1, 'type': 'MISC', 'global_pos': [41, 41], 'index': '0_0'}]","[{'pos': [25, 26], 'type': 'TIME', 'sent_id': 1, 'name': '2010', 'global_pos': [43, 43], 'index': '6_0'}]",P577,publication date,When was Loud Tour published?,0,{},1.0,"Loud Tour The Loud Tour was the fourth overall and third world concert tour by Barbadian recording artist Rihanna . Performing in over twenty countries in the A

In [6]:
df = df_raw.copy()
# df = df[df["evidence_sent_ids"].str.len() == 1]  # 1 Evidence
# df = df[df["evidence_sents"].str.len() == 1]  # 1 Evidence Sentence
# df = df[df["head_entity_in_evidence"].str.len() >= 1]  # 1 Head in Evidence
# df = df[df["head_entity_names"].str.len() == 1]  # All heads have the same name
# df = df[df[""]]
print(len(df))

### Filter Repeated Labels (Only 1 h->t)
# def not_repeated_label(label, labels):
#     repeat_count = 0
#     for l in labels:
#         if l['h'] == label['h'] and l['t'] == label['t']:
#             repeat_count += 1
#     return True if repeat_count == 1 else False
# df["repeated_label"] = df.apply(lambda r: not_repeated_label(r["label"], r["labels"]), axis=1)
# df = df[df["repeated_label"]]

# df = df.sample(250, random_state=0)
# print(len(df))

def flatten(xss):
    return [x for xs in xss for x in xs]

# df["sents_complete"] = df["sents"].apply(lambda x: " ".join(flatten(x)))
# df["sents_evidence_single"] = df["evidence_sents"].apply(lambda x: " ".join(flatten(x)))
# df["sents_evidence_redundant"] = df.apply(lambda r: r["sents_evidence_single"] + " " + df[df['title'] != r['title']].sample(1, random_state=0)["sents_complete"].values[0], axis=1)
# df["sents_evidence_doc"] = df.apply(lambda r: r["sents_evidence_single"] + " " + r["sents_complete"], axis=1)

sents_cols = []
### evidence > evidence + redundant long text
# sents_cols.append(f"sents_evidence_single")
# sents_cols.append(f"sents_evidence_redundant")  # 35
# sents_cols.append(f"sents_evidence_doc")  # 17

# print(df.iloc[0]["sents_evidence_redundant"])
df.sample(2)

7170


,query_id,query,gold_docs,gold_docs_text,scores_stats,scores_gold,scores_1000,predicted_docs_text_10,id,title,vertexSet,labels,sents,split,label,label_idx,head_entity,tail_entity,head_entity_names,tail_entity_names,head_entity_longest_name,tail_entity_longest_name,head_entity_types,tail_entity_types,evidence_sent_ids,evidence_sents,head_entity_in_evidence,tail_entity_in_evidence,relation,relation_name,query_question,duplicate_titles_len,duplicate_titles,hit_rank,gold_doc,gold_doc_title,gold_doc_text,gold_doc_score,pred_doc,pred_doc_title,pred_doc_text,pred_doc_score,gold_doc_len,pred_doc_len,query_decompx_tokens,query_decompx_tokenizer_word_ids,query_decompx_cls_or_mean_pooled,query_decompx_tokens_dot_scores,query_decompx_decompx_last_layer_pooled,gold_doc_decompx_tokens,gold_doc_decompx_tokenizer_word_ids,gold_doc_decompx_cls_or_mean_pooled,gold_doc_decompx_tokens_dot_scores,gold_doc_decompx_decompx_last_layer_pooled,pred_doc_decompx_tokens,pred_doc_decompx_tokenizer_word_ids,pred_doc_decompx_cls_or_mean_pooled,pred_doc_decompx_tokens_dot_scores,pred_doc_decompx_decompx_last_layer_pooled
5667,validation9338,Which country is Emancipation Proclamation associated with?,[Washington Place (West Virginia)],"{'Washington Place (West Virginia)': {'text': 'Washington Place ( William Washington House ) is one of the first homes built by freed slaves after the Emancipation Proclamation of 1863 in Hampshire County , West Virginia , United States . Washington Pl...","{'len': 1000, 'max': 377.0677490234375, 'min': 369.3297119140625, 'std': 0.9824244489004268, 'mean': 370.3620311279297, 'median': 370.084716796875}",{'Washington Place (West Virginia)': None},"{'Proclamation of the Irish Republic': 377.0677490234375, 'Emancipation': 376.9818420410156, 'First Reading of the Emancipation Proclamation of President Lincoln': 376.6058349609375, 'Juneteenth': 376.1060485839844, 'Foreign relations of Lesotho': 374....","{'Proclamation of the Irish Republic': {'text': 'The Proclamation of the Republic ( ) , also known as the 1916 Proclamation or the Easter Proclamation , was a document issued by the Irish Volunteers and the Irish Citizen Army during the Easter Rising i...",validation9338,Washington Place (West Virginia),"[[{'pos': [0, 2], 'type': 'LOC', 'sent_id': 0, 'name': 'Washington Place', 'global_pos': [0, 0], 'index': '0_0'}, {'name': 'William Washington House', 'pos': [3, 6], 'sent_id': 0, 'type': 'LOC', 'global_pos': [3, 3], 'index': '0_1'}, {'pos': [0, 2], 't...","[{'r': 'P17', 'h': 1, 't': 5, 'evidence': [0]}, {'r': 'P131', 'h': 3, 't': 4, 'evidence': [0]}, {'r': 'P17', 'h': 3, 't': 5, 'evidence': [0]}, {'r': 'P150', 'h': 4, 't': 3, 'evidence': [0]}, {'r': 'P131', 'h': 4, 't': 5, 'evidence': [0]}, {'r': 'P17', ...","[[Washington, Place, (, William, Washington, House, ), is, one, of, the, first, homes, built, by, freed, slaves, after, the, Emancipation, Proclamation, of, 1863, in, Hampshire, County, ,, West, Virginia, ,, United, States, .], [Washington, Place, was,...",validation,"{'r': 'P17', 'h': 1, 't': 5, 'evidence': [0]}",0,"[{'name': 'Emancipation Proclamation', 'pos': [19, 21], 'sent_id': 0, 'type': 'MISC', 'global_pos': [19, 19], 'index': '1_0'}]","[{'name': 'United States', 'pos': [30, 32], 'sent_id': 0, 'type': 'LOC', 'global_pos': [30, 30], 'index': '5_0'}]",{Emancipation Proclamation},{United States},Emancipation Proclamation,United States,{MISC},{LOC},[0],"[[Washington, Place, (, William, Washington, House, ), is, one, of, the, first, homes, built, by, freed, slaves, after, the, Emancipation, Proclamation, of, 1863, in, Hampshire, County, ,, West, Virginia, ,, United, States, .]]","[{'name': 'Emancipation Proclamation', 'pos': [19, 21], 'sent_id': 0, 'type': 'MISC', 'global_pos': [19, 19], 'index': '1_0'}]","[{'name': 'United States', 'pos': [30, 32], 'sent_id': 0, 'type': 'LOC', 'global_pos': [30, 30], 'index': '5_0'}]",P17,country,Which country is Emancipation Proclamation associated with?,0,{},inf,"Washington Place (West Vir

In [7]:
df["head_entity_types"].value_counts()

head_entity_types
{PER}         2373
{LOC}         1997
{MISC}        1803
{ORG}          978
{LOC, ORG}       7
{TIME}           4
{PER, ORG}       3
{NUM}            3
{PER, LOC}       2
Name: count, dtype: int64

In [12]:
df["head_has_digit"] = df.apply(lambda r: any(char.isdigit() for char in r["head_entity_longest_name"][0]), axis=1)
print(len(df[df["head_has_digit"]]))
df[df["head_has_digit"]].sample(2)

49


,query_id,query,gold_docs,gold_docs_text,scores_stats,scores_gold,scores_1000,predicted_docs_text_10,id,title,vertexSet,labels,sents,split,label,label_idx,head_entity,tail_entity,head_entity_names,tail_entity_names,head_entity_longest_name,tail_entity_longest_name,head_entity_types,tail_entity_types,evidence_sent_ids,evidence_sents,head_entity_in_evidence,tail_entity_in_evidence,relation,relation_name,query_question,duplicate_titles_len,duplicate_titles,hit_rank,gold_doc,gold_doc_title,gold_doc_text,gold_doc_score,pred_doc,pred_doc_title,pred_doc_text,pred_doc_score,gold_doc_len,pred_doc_len,query_decompx_tokens,query_decompx_tokenizer_word_ids,query_decompx_cls_or_mean_pooled,query_decompx_tokens_dot_scores,query_decompx_decompx_last_layer_pooled,gold_doc_decompx_tokens,gold_doc_decompx_tokenizer_word_ids,gold_doc_decompx_cls_or_mean_pooled,gold_doc_decompx_tokens_dot_scores,gold_doc_decompx_decompx_last_layer_pooled,pred_doc_decompx_tokens,pred_doc_decompx_tokenizer_word_ids,pred_doc_decompx_cls_or_mean_pooled,pred_doc_decompx_tokens_dot_scores,pred_doc_decompx_decompx_last_layer_pooled,head_has_digit
3426,test15681,Which administrative territorial entity is 19th Route Army located in?,[Route Army],"{'Route Army': {'text': 'A Route Army ( 路軍/路军 ) , was a type of military organization during the Chinese Republic , and usually exercised command over two or more corps or a large number of divisions or independent brigades . It was a common formation ...","{'len': 1000, 'max': 378.88409423828125, 'min': 367.6964111328125, 'std': 1.1245739467384355, 'mean': 368.82406903076173, 'median': 368.4997253417969}",{'Route Army': 378.88409423828125},"{'Route Army': 378.88409423828125, 'Nineteenth Air Force': 375.5193786621094, 'Northern Territory Force': 374.8721923828125, 'Southern Army (Home Forces)': 374.06610107421875, 'Military Road-Cadron Segment': 373.9791564941406, 'Army of Württemberg': 37...","{'Route Army': {'text': 'A Route Army ( 路軍/路军 ) , was a type of military organization during the Chinese Republic , and usually exercised command over two or more corps or a large number of divisions or independent brigades . It was a common formation ...",test15681,Route Army,"[[{'pos': [1, 3], 'type': 'ORG', 'sent_id': 0, 'name': 'Route Army', 'global_pos': [1, 1], 'index': '0_0'}, {'pos': [0, 3], 'type': 'ORG', 'name': '8th Route Army', 'sent_id': 3, 'global_pos': [93, 93], 'index': '0_1'}, {'pos': [0, 3], 'type': 'ORG', '...","[{'r': 'P1336', 'h': 2, 't': 8, 'evidence': []}, {'r': 'P150', 'h': 2, 't': 9, 'evidence': [1, 4]}, {'r': 'P150', 'h': 2, 't': 12, 'evidence': [1, 5]}, {'r': 'P17', 'h': 4, 't': 2, 'evidence': [1]}, {'r': 'P17', 'h': 7, 't': 2, 'evidence': [1, 3]}, {'r...","[[A, Route, Army, (, 路軍/路军, ), ,, was, a, type, of, military, organization, during, the, Chinese, Republic, ,, and, usually, exercised, command, over, two, or, more, corps, or, a, large, number, of, divisions, or, independent, brigades, .], [It, was, a...",test,"{'h': 0, 't': 2, 'r': 'P131', 'evidence': [1, 2, 3, 4, 5]}",65,"[{'pos': [1, 3], 'type': 'ORG', 'sent_id': 0, 'name': 'Route Army', 'global_pos': [1, 1], 'index': '0_0'}, {'pos': [0, 3], 'type': 'ORG', 'name': '8th Route Army', 'sent_id': 3, 'global_pos': [93, 93], 'index': '0_1'}, {'pos': [0, 3], 'type': 'ORG', 'n...","[{'name': 'China', 'pos': [6, 7], 'sent_id': 1, 'type': 'LOC', 'global_pos': [43, 43], 'index': '2_0'}]","{Route Army, 19th Route Army, Route Armies, 8th Route Army, 29th Route Army}",{China},19th Route Army,China,{ORG},{LOC},"[1, 2, 3, 4, 5]","[[It, was, a, common, formation, in, China, prior, to, the, Second, Sino, -, Japanese, War, but, was, discarded, as, a, formation, type, by, the, National, Revolutionary, Army, after, 1938, (, other, than, the, 8th, Route, Army, ), ,, in, favor, of, th...","[{'pos': [0, 3], 'type': 'ORG', 'name': '8th Route Army', 'sent_id': 3, 'global_pos': [93, 93], 'index': '0_1'}, {'pos': [0, 3], 'type': 'ORG', 'name': '19th Route Army', 'sent_id': 4, 'global